데이터 증강을 하기 위해 백번역 기법을 사용하였다. 전처리 한 질문을 먼저 한국어에서 영어로 번역 한 후, 다시 영어 문장을 한국어로 역번역하여 새로운 질문을 생성하는 방식이다.
이 과정에서 MBart 모델과 토크나이저를 활용하여 번역을 진행하였다. 이때 원문의 의미를 유지하면서 표현 방식이나 단어 선택, 어미 등만 달라지도록 하여 데이터를 증강했다.
다만, 역번역 과정에서 의미나 뉘앙스가 조금씩 달라질 수 있기 때문에 결과 문장들을  검토하는 작업도 수행했다.



In [ ]:
!pip install transformers torch sentencepiece

transformers, torch, sentencepiece 라이브러리를 설치한다. 모델 불러오기와 토크나이징에 필요한 패기지를 준비하는 단계이다.

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

MBart 모델과 토크나이저 클래스를 불러온다.  MBartForConditionalGeneration은 조건부 문장 생성을 위한 모델이고, MBart50TokenizerFast는 빠른 토크나이저 클래스이다.

In [ ]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)


facebook/mbart-large-50-many-to-many-mmt' 모델을 불러오는 코드이다.
모델 이름을 지정하고 토그나이저와 모델을 from_pretrained() 메서드를 사용하여 로드한다.


In [2]:

def back_translate(sentence):
->백번역을 위한 함수이다. 두 번의 번역을 통해서 입력 문장의 표현을 다양화해준다.

    tokenizer.src_lang = "ko_KR"
    encoded_ko = tokenizer(sentence, return_tensors="pt")
    translated_ids = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]

    tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(translated_text, return_tensors="pt")
    back_translated_ids = model.generate(**encoded_en, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
    final_sentence = tokenizer.batch_decode(back_translated_ids, skip_special_tokens=True)[0]


IndentationError: expected an indented block after function definition on line 1 (<ipython-input-2-f5048a161c15>, line 2)

백번역을 위한 함수이다. 두 번의 번역을 통해서 입력 문장의 표현을 다양화해준다.


tokenizer.src_lang = "ko_KR"
    encoded_ko = tokenizer(sentence, return_tensors="pt")
    translated_ids = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
    translated_text = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)[0]


->한국어 문장을 영어로 번역하는 단계이다. 토그나이저의 src_lang을 "ko_KR"로 설정해 번역할 문장을 한국어로 지정한다.
tokenizer()를 사용해서 문장을 인코딩하고 PyTorch 텐서 형식으로 변환한다.
model.generate()를 호출할 때 forced_bos_token_id를 영어 시작 토큰("en_XX")으로 지정해 영어 문장을 생성하게 한다.
batch_decode()를 통해 생성된 토큰 시퀀스를 텍스트로 디코딩한다.



tokenizer.src_lang = "en_XX"
    encoded_en = tokenizer(translated_text, return_tensors="pt")
    back_translated_ids = model.generate(**encoded_en, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
    final_sentence = tokenizer.batch_decode(back_translated_ids, skip_special_tokens=True)[0]

    
->번역된 영어 문장을 다시 한국어로 역번역하는 단계이다.
src_lang을 "en_XX"로 설정하여 번역할 문장으로 영어로 지정한다.
번역된 텍스트를 토크나이저로 인코딩한 후, forced_bos_token_id를 "ko_KR"로 지정해 한국어 문장을 생성하게 한다.
생성된 토큰 시퀀스를 디코딩하여 최종 한국어 문장을 추출한다.


In [3]:
df = pd.read_excel("input.xlsx")
df["back_translated"] = df["question"].apply(back_translate)

NameError: name 'pd' is not defined

엑셀 파일 'input.xlsx'를 읽어 'question' 열에 있는 질문들에 백번역 함수를 적용

In [ ]:
df.to_excel("output.xlsx", index=False)

백번역 결과가 포함된 데이터프레임을 'output.xlsx' 파일로 저장한다.